In [4]:
from requests_html import HTMLSession #最新的爬虫库，
import re #强大的正则
import time
import datetime
import pandas as pd
import tushare as ts
session = HTMLSession()#发起会话
urls =[ 'http://www.ddxzx.com/ygetnewallddxpm.php?zf=0&ddx=0&ddy=0&page={}&pagenum=20&orderby=5&isdesc=1&d=sz'.format(str(i)) for i in range(1, 3)]
mylist = [] #前面通过for循环实现翻页
for url in urls: 
    try:
        r=session.get(url) #挨个HTML访问

        r = list(r)      
        r = r[:-3]
        r = str(r)#正则匹配前需转换成字符串
        r = re.findall(r"\d{6}",r)  #正则匹配6位数字
        mylist=mylist+r #列表循环增加
    #    print( mylist)
        time.sleep(3) 
    except:
        pass
df = pd.DataFrame(mylist,columns = ["code"])
#df = df.to_excel("/home/wangd/ddx.xlsx")
dfyl = ts.get_growth_data(2018,3)#考察盈利状况
df1 = dfyl.query("mbrg>5.00 and nprg>5.0")#筛选增长性好的

#dfdq = ts.get_today_all()#获取当前行情
#df2 = dfdq.query("changepercent<-2.000")#设定当前价格的选股标准
df3 = pd.merge(df1,df,on="code",how="inner")#再次筛选
df3 = df3.drop_duplicates()
print(df3)
#df3.to_excel("/home/wangd/ddx和增长性.xlsx")

code = df3["code"]
today = datetime.date.today()
oneday = datetime.timedelta(days=20)
beging = str(today-oneday)
for i in code:  #逐个下载历史行情，从技术形态层面分析
    try:
   
        df = ts.get_hist_data('%06s' % i,start='%s' % beging)   #%06d,表示数字是6位数，如不够就自动加0到前面
   
    
        if df.iloc[0,2] < df["close"].mean()*1.05:#当日收盘价小于历史20日均价5%
            
    
            print(i)
            
           
            
        else:
            pass
    except:
        pass




    
   


[Getting data:]############################################################      code  name      mbrg      nprg       nav      targ      epsg       seg
0   600216  浙江医药   30.2793  282.6640    8.2151    6.3883  285.7143   10.7912
1   600789  鲁抗医药   25.9807  192.2913   54.3728   36.0626  197.3333   55.4308
2   300732   设研院   19.3878   44.8866  126.5952   74.4161  -39.7321  126.5952
3   300495  美尚生态   20.3493   37.1169   13.9768   39.3759   36.6771   12.6266
4   300148  天舟文化   32.6889   36.9811    1.4834    6.7902   37.5000    1.3529
5   002796  世嘉科技  106.7765   22.1914  187.4359  187.5745  -10.3448  185.0850
6   603080  新疆火炬   13.5896   17.1180  101.2477   80.8694  -12.7660  101.2477
7   002921  联诚精密    9.2051   10.9781   51.5019   31.0849  -19.1639   51.8773
8   300674  宇信科技   20.1384   10.9132       NaN       NaN   15.0000       NaN
9   002472  双环传动   26.3445    5.9161   16.0036   37.9413    3.7037   15.9384
10  603103  横店影视   13.9703    5.9152   12.5452    6.7462   -6.5789   12.5452
3